In [2]:
import numpy as np
import sklearn as sk
import gower
from sklearn import metrics
from sklearn import feature_selection
from sklearn import decomposition
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("cleaned_missing_kept_v1.csv")

In [4]:
df.drop([
    "CREATED_ON",
    "LAST_LOGIN",
    "SERVICE_X_DATE",
    "PACKAGE_CHANGE_DATE",
    "SUPERUSER_LAST_LOGIN",
    "ACCOUNTANT_LAST_LOGIN"
    ], axis=1, inplace=True)

In [5]:
df[["LINE_OF_BUSINESS", "SIZE_CODE", "STATUS_CODE"]] = df[["LINE_OF_BUSINESS", "SIZE_CODE", "STATUS_CODE"]].astype(str).replace('nan',np.nan)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27423 entries, 0 to 27422
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   LINE_OF_BUSINESS              20656 non-null  object 
 1   SIZE_CODE                     21509 non-null  object 
 2   PACKAGE_TYPE                  27394 non-null  object 
 3   PREVIOUS_PACKAGE_TYPE         26277 non-null  object 
 4   SERVICE_1_COUNT               27419 non-null  float64
 5   SERVICE_2_COUNT               27419 non-null  float64
 6   SERVICE_2_TOTAL               27419 non-null  float64
 7   SERVICE_3_COUNT               27419 non-null  float64
 8   SUPERUSER_ID                  27423 non-null  float64
 9   ACCOUNTANT_ID                 27423 non-null  float64
 10  GROSS_INCOME                  19131 non-null  float64
 11  STATUS_CODE                   21511 non-null  object 
 12  RATING_CODE                   20680 non-null  float64
 13  C

In [15]:
print(df.var(axis=0).mean())
print(df.var(axis=0))

0.1355534696853568
SERVICE_1_COUNT                 0.000319
SERVICE_2_COUNT                 0.000061
SERVICE_2_TOTAL                 0.000053
SERVICE_3_COUNT                 0.000393
SUPERUSER_ID                    0.237324
ACCOUNTANT_ID                   0.040178
GROSS_INCOME                    0.000213
RATING_CODE                     0.859936
CREATED_ON_ACTIVE               0.148510
LAST_LOGIN_ACTIVE               0.246077
SERVICE_X_DATE_ACTIVE           0.048153
PACKAGE_CHANGE_DATE_ACTIVE      0.040078
SUPERUSER_LAST_LOGIN_ACTIVE     0.248782
ACCOUNTANT_LAST_LOGIN_ACTIVE    0.027670
dtype: float64


C:\Users\anonym\AppData\Local\Temp/ipykernel_20148/3486316742.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df.var(axis=0).mean())
C:\Users\anonym\AppData\Local\Temp/ipykernel_20148/3486316742.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df.var(axis=0))


Based on the variances, we can drop two very homogenous columns: `SERVICE_2_COUNT` and `SERVICE_2_TOTAL`.

In [ ]:
df.drop(["SERVICE_2_COUNT", "SERVICE_2_TOTAL"], axis=1, inplace=True)

In [24]:
print(len(df))
for col in df.columns:
    uvals = df[col].unique()
    print(col)
    print(len(uvals))
    if len(uvals) < 50:
        print(uvals)
    print()

27423
LINE_OF_BUSINESS
475

SIZE_CODE
19
['91406695.0' '91406690.0' '91406692.0' '91406694.0' '91406683.0' nan
 '91406697.0' '91406693.0' '91406696.0' '91406698.0' '91406688.0'
 '91406682.0' '91406685.0' '91406699.0' '91406687.0' '91406684.0'
 '91406689.0' '91406686.0' '91406691.0']

PACKAGE_TYPE
11
['Free_tier_0' 'Hibernated' 'paid_tier_1' 'paid_tier_2' 'Hibernated_2'
 'Freemium' 'Free_tier_1' nan 'Free_tier_2' 'Free_tier_3'
 'Free_tier_promo']

PREVIOUS_PACKAGE_TYPE
14
['Free_tier_0' 'Hibernated' 'paid_tier_1' 'S' 'paid_tier_2' 'Hibernated_2'
 'Freemium' 'M' 'Free_tier_1' 'Free_tier_2' nan 'ExleX' 'Free_tier_3'
 'Free_tier_promo']

SERVICE_1_COUNT
461

SERVICE_2_COUNT
94

SERVICE_2_TOTAL
770

SERVICE_3_COUNT
506

SUPERUSER_ID
2
[1. 0.]

ACCOUNTANT_ID
2
[0. 1.]

GROSS_INCOME
16631

STATUS_CODE
11
['91406700.0' '91406713.0' nan '91406701.0' '91406705.0' '91406710.0'
 '91406711.0' '91406702.0' '91406714.0' '91406708.0' '91406715.0']

RATING_CODE
5
[ 4.  1.  2. nan  3.]

CREATED_ON_ACTIV

In [25]:
#gdist = gower.gower_matrix(df)
#np.save("gower_dist_v3.npy", gdist)